In [1]:
"""
Generate speaker embeddings and metadata for training
"""
import os
import pickle
import numpy as np
import torch
from factory.LstmDV import LstmDV
#from factory.MetaDV import MetaDV

In [2]:
device = "cuda:0"

In [3]:
#C = MetaDV(80,256).to(device)
C = LstmDV(80).to(device)
C.load_state_dict(torch.load("../model/static/dv_vctk80.pth", map_location=device))

<All keys matched successfully>

In [4]:
# HOW MANY DIFFERENT CONTENT OF VOICE IN YOUR DATA
num_uttrs = 200
errorment = 50
len_crop = 128

In [5]:
# Directory containing mel-spectrograms
rootDir = './train_spmel_vctk80'
dirName, subdirList, _ = next(os.walk(rootDir))
print('Found directory: %s' % dirName)

Found directory: ./train_spmel_vctk80


In [6]:
def pad_along_axis(array: np.ndarray, target_length: int, axis: int = 0):
    pad_size = target_length - array.shape[axis]
    if pad_size <= 0:
        return array
    npad = [(0, 0)] * array.ndim
    npad[axis] = (0, pad_size)
    return np.pad(array, pad_width=npad, mode='constant', constant_values=0)

In [7]:
speakers = []
for speaker in sorted(subdirList):
    print('Processing speaker: %s' % speaker)
    utterances,embs  = [],[]
    utterances.append(speaker)
    _, _, fileList = next(os.walk(os.path.join(dirName,speaker)))
    # make speaker embedding
    assert len(fileList) >= num_uttrs
    idx_uttrs = np.random.choice(len(fileList), size=num_uttrs, replace=False)
    for i in range(errorment):
        tmp = np.load(os.path.join(dirName, speaker, fileList[idx_uttrs[i]]))
        # pad if the current one is too short   
        if tmp.shape[0] < len_crop:
            tmp = pad_along_axis(tmp,len_crop)
            melsp = torch.from_numpy(tmp[np.newaxis,:, :]).cuda()
        else:
            melsp = torch.from_numpy(tmp[np.newaxis,:, :]).cuda()
        emb = C(melsp)[1]
        embs.append(emb.detach().squeeze().cpu().numpy())    
           
    utterances.append(np.mean(embs, axis=0))
    # create file list
    want_uttr = 0
    for fileName in sorted(fileList):
        tmp = np.load(os.path.join(dirName, speaker,fileName))
        # 不足 100 的丟掉
        if tmp.shape[0] < 100:
            print(tmp.shape)
            continue
        elif want_uttr < num_uttrs:
            utterances.append(os.path.join(speaker,fileName))
            want_uttr += 1
        else:
            continue
    speakers.append(utterances)

Processing speaker: p225
Processing speaker: p226
Processing speaker: p227
Processing speaker: p228
Processing speaker: p229
Processing speaker: p230
Processing speaker: p231
Processing speaker: p232
Processing speaker: p233
Processing speaker: p234
Processing speaker: p236
Processing speaker: p237
Processing speaker: p238
Processing speaker: p239
Processing speaker: p240
Processing speaker: p241
Processing speaker: p243
Processing speaker: p244
Processing speaker: p245
Processing speaker: p246
Processing speaker: p247
Processing speaker: p248
Processing speaker: p249
Processing speaker: p250
Processing speaker: p251
Processing speaker: p252
Processing speaker: p253
Processing speaker: p254
Processing speaker: p255
Processing speaker: p256
Processing speaker: p257
Processing speaker: p258
Processing speaker: p259
Processing speaker: p260
Processing speaker: p261
Processing speaker: p262
Processing speaker: p263
Processing speaker: p264
Processing speaker: p265
Processing speaker: p266


In [8]:
with open(os.path.join(rootDir, 'train_lstm.pkl'), 'wb') as handle:
    pickle.dump(speakers, handle)